In [1]:

include("../general_modules/channels.jl")
include("../general_modules/mesh.jl")
using .channels
using .mesh
using JSON

In [3]:
data = JSON.parsefile("test.json")
Systems_params= data["Systems"]
mesh_params= data["Mesh"]
α= α3b(0.5,0.5,1,Systems_params["lmax"],Systems_params["lmin"],Systems_params["λmax"],Systems_params["λmin"],Systems_params["s1"],Systems_params["s2"],Systems_params["s3"],Systems_params["t1"],Systems_params["t2"],Systems_params["t3"],Systems_params["MT"])
θi, dθi, xi, dxi, yi, dyi= initialmesh(mesh_params["nθ"],mesh_params["nx"],mesh_params["ny"],Float64(mesh_params["xmax"]),Float64(mesh_params["ymax"]),Float64(mesh_params["alpha"]))

For J=0.5 T=0.5 parity=1 Number of channels: 8


([0.010793571159988469, 0.05659276429335236, 0.13786183772187122, 0.25271446970529854, 0.3984609095574594, 0.5716855414653129, 0.768328316649816, 0.9837801753398278, 1.2129911485257514, 1.450588748496007, 1.6910039050937862, 1.9286015050640417, 2.1578124782499652, 2.373264336939977, 2.56990711212448, 2.7431317440323335, 2.8888781838844944, 3.0037308158679217, 3.0849998892964408, 3.130799082429805], [0.027668017714319135, 0.06377657679306871, 0.09844502331593069, 0.13081079977613538, 0.16011145779868483, 0.18565953665239512, 0.20685602955658772, 0.22320404656916054, 0.2343203792081908, 0.239944459410423, 0.239944459410423, 0.2343203792081908, 0.22320404656916054, 0.20685602955658772, 0.18565953665239512, 0.16011145779868483, 0.13081079977613538, 0.09844502331593069, 0.06377657679306871, 0.027668017714319135], [0.021207217853561045, 0.11187676267955583, 0.27556261305361035, 0.5132871869296085, 0.8265233725878167, 1.2172749571705945, 1.6881545946966547, 2.2424901542124482, 2.8844726104096

In [5]:
using Revise
include("Gcoefficient.jl")
using .Gcoefficient

initialY(λmax, lmax, nθ, nx, ny, θi, xi, yi, P::Char)
return Yλαout, Ylαin, Yλαin

In [10]:
Yλαout, Ylαin, Yλαin = initialY(Systems_params["λmax"], Systems_params["lmax"], mesh_params["nθ"],mesh_params["nx"],mesh_params["ny"], θi, xi, yi, '+')

UndefVarError: UndefVarError: `sphericalharmonic` not defined in `Main.Gcoefficient`
Suggestion: check for spelling errors or missing imports.